In [ ]:
!pip install accelerate -U -q
!pip install git+https://github.com/Bots-Avatar/ExplainitAll -q

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from explainitall.fast_tuning.trainers.HMMTrainer import GPT2HMMDataProcessor
from explainitall.fast_tuning.generators import GeneratorWithExpert
from explainitall.fast_tuning.generators import MCExpert
from explainitall.fast_tuning.generators import MCModel
from explainitall.fast_tuning.trainers.ProjectionTrainer import GPTProjectionTrainer
from explainitall.fast_tuning.ExpertBase import ExpertModel
import numpy as np
import torch
import re

# Датасет
Создание и сохранение датасета (полстранички из Википедии)

---

In [ ]:
text_data = '''Электро́нная лампа, радиолампа — электровакуумный прибор (точнее, вакуумный электронный прибор), работающий за счёт управления интенсивностью потока электронов, движущихся в вакууме или разрежённом газе между электродами[1].
Радиолампы массово использовались в XX веке как основные элементы радиоаппаратуры, так как позволяют выпрямлять ток, усиливать, генерировать электрические сигналы и т. п. С появлением полупроводниковых приборов (диодов, транзисторов) радиолампы стали вытесняться из радиоаппаратуры, так как полупроводниковые приборы оказались значительно компактнее и экономичнее. В настоящее время радиолампы встречаются там, где полупроводниковые аналоги получаются дороже или сложнее в изготовлении, например в качестве мощного генератора радиоволн в микроволновой печи используется радиолампа — магнетрон. Также радиолампы традиционно используются в некоторых видах аудиоаппаратуры, позиционируемой как высококачественная (high-end).
Электронные лампы, предназначенные для освещения (лампы-вспышки, ксеноновые лампы, ртутные и натриевые лампы), радиолампами не называются и обычно относятся к классу осветительных приборов.
Электронно-лучевые приборы основаны на тех же принципах, что и радиолампы, но, помимо управления интенсивностью электронного потока, также управляют распределением электронов в пространстве и потому выделяются в отдельную группу. Также в отдельную группу выделяют СВЧ-электровакуумные приборы, основанные на взаимодействии электронного потока с электромагнитным полем в таких приборах как магнетрон, клистрон и др.
Самая простая радиолампа имеет колбу, внутри которой размещены два электрода — катод и анод. Катод разогревается от источника тока до температуры, когда из него вследствие термоэлектронной эмиссии могут вылетать электроны, и свободно перемещаться внутри вакуума колбы. Электроны имеют отрицательный заряд, и если на второй электрод, анод, будет подан положительный потенциал, электроны устремятся к аноду, попадут в него и создадут ток в цепи анод-катод. Если на анод подать отрицательный потенциал, то электроны имея одноимённый заряд будут отталкиваться от анода, и ток в цепи протекать не будет. Такая простая радиолампа называется кенотрон и пригодна для выпрямления переменного тока в постоянный ток, так как проводит ток только в одном направлении.
Более сложная радиолампа — триод устроена так же, но имеет ещё и третий электрод — сетку, расположенную между анодом и катодом. Если потенциал на сетке отсутствует, а на аноде потенциал положительный, то все электроны вылетевшие с катода устремляются к аноду и создают ток в цепи анода. Если подать на сетку небольшой отрицательный потенциал, то она своим полем сможет отклонять часть электронов на пути к аноду, тем самым уменьшая ток анода. Чем выше отрицательный потенциал на сетке — тем бо́льшая часть электронов будет отклонена, тем меньше ток анода. Если подать на сетку достаточно большой отрицательный потенциал — то лампа окажется «заперта» — ток в цепи анода прекратится. Такая лампа может работать как усилитель, если подать на сетку слабый электрический сигнал, то он вызовет синхронные изменения тока анода, причем на ощутимо бо́льшие величины.
Различные усложнения конструкции лампы — применение катода косвенного накала, введение дополнительных сеток, изменение формы колбы или введение в него небольшого количества газа улучшают одни параметры лампы, ухудшая другие, но основной принцип работы радиолампы не меняется — управление потоком электронов от катода к аноду при помощи электрического поля сеток.
Существенным недостатком радиоламп является её размер и необходимость постоянно тратить энергию на поддержание катода в нагретом состоянии (кроме ламп с холодным катодом).
Вакуумные электронные лампы с подогревным катодом
В результате термоэлектронной эмиссии электроны покидают катод.
Под воздействием напряжения между анодом и катодом электроны достигают анода и образуют анодный ток во внешней цепи.
С помощью дополнительных электродов (сеток) осуществляется управление электронным потоком путём подачи на эти электроды электрических напряжений.
Электронная лампа RCA '808'
В вакуумных электронных лампах наличие газа ухудшает характеристики лампы.
Газоразрядные электронные лампы
В СССР и в России традиционно выделяются в отдельный класс ионных приборов в отличие от вакуумных ламп. Основной ток проводимости в этих устройствах вызван потоком ионов в газе, наполняющем лампу. Ионизация газа может вызываться соударениями атомов или молекул газа с электронами, как и в вакуумных лампах эмиттируемыми накалённым катодом, а может создаваться самоподдерживающимся разрядом в разреженном газе за счёт ускорения ионов электрическим полем. Как правило, такие лампы используются либо в низкочастотных и импульсных генераторах (тиратроны), либо в схемах управляемых выпрямителей, часто с высокими выходными токами — схемы на игнитронах.
Типы газоразрядных электронных ламп:
неоновая лампа;
газоразрядный стабилитрон;
ионный разрядник;
тиратрон;
игнитрон.
Неоновая лампа
Неоновая лампа — разновидность газоразрядного прибора тлеющего разряда, представляет собой стеклянный баллона в котором располагаются два электрода. Баллон наполнен инертным газом (неоном) при небольшом давлении. Электроды изготавливаются из металла, например никеля, и могут быть различной формы (два цилиндрических, два плоских и др.)
Неоновые лампы излучают оранжево-красный свет небольшой интенсивности и используются, в основном, как индикаторные. Неоновую лампу подключают к источнику напряжения последовательно с ограничительным резистором, иначе разряд сразу переходит в дуговой и лампа выходит из строя.
Стабилитрон
Газоразрядный стабилитрон представляет собой стеклянный баллон, в котором находятся два электрода — катод и анод. Катод имеет форму цилиндра с большой поверхностью, анод — стержень, расположенный вдоль оси катода. Внутренняя поверхность катода активируется. Баллон наполняется аргоном, неоном или смесью газов при давлении в несколько десятков миллиметров ртутного столба. Благодаря большой поверхности катода, напряжение между электродами при значительных изменениях тока тлеющего разряда остается неизменным.
Параметрами стабилитрона являются: напряжение зажигания, напряжение горения, минимальный и максимальный ток. Величина напряжения стабилизации зависит от вида газа и материала катода, которым наполнен баллон.
Стабилитрон с коронным разрядом
Кроме стабилитронов с тлеющим разрядом, описанных выше, существуют стабилитроны с коронным разрядом. Устройство данных стабилитронов схоже со стабилитронами тлеющего разряда. Баллон наполняется водородом при низком давлении. Стабилитроны с коронным разрядом имеют в несколько раз более высокие значения напряжения горения, и позволяют стабилизировать напряжение порядка 300—1000 В и более. Однако ток, проходящий через такой стабилитрон в сотни раз меньше чем у стабилитронов с тлеющим разрядом[2].
'''

# Загрузка основной модели

Загрузка модели "rugpt3large_based_on_gpt2".

In [ ]:
path = 'ai-forever/rugpt3large_based_on_gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(path)
gpt = GPT2LMHeadModel.from_pretrained(path)
gpt.to('cuda')

# Создание и тестирование эксперта на базе марковской цепи

In [ ]:
%%time
text = text_data.replace('\n', ' ').replace('  ', ' ')
hmm_expert_creator = GPT2HMMDataProcessor(tokenizer)
tokens = hmm_expert_creator.get_data_1(re.split('\.|,', text))
data = hmm_expert_creator.create_data(tokens)
hmm_expert_states = hmm_expert_creator.train(data)

bias_mask = set(tokens.flatten()) # Маска для смещения (словарь токенов)

l_v = tokenizer.vocab_size
mc_model = MCModel.MarkovModel(l_v, data['x_encoder'], data['y_decoder'], model=hmm_expert_states)
hmm_expert = MCExpert.MCExpert(mc_model)
l_v

In [ ]:
# Простой(жадный) генератор для эксперта
def generate(input_text, expert, tokenizer, len_t = 30):
  t = tokenizer.encode(input_text)
  tokens_expert = t

  for i in range(len_t):
    input_tok = tokens_expert
    outp_token = np.argmax(expert.get_bias(input_tok))
    tokens_expert.append(outp_token)

  return tokenizer.decode(tokens_expert)

In [ ]:
generate("Лампа",hmm_expert, tokenizer)

# Обучение слоев проекций основной сети


In [ ]:
%%time
trainer = GPTProjectionTrainer(gpt, tokenizer)
trainer.train([text_data],bias_mask, num_train_epochs = 60)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss


CPU times: user 55.8 s, sys: 130 ms, total: 55.9 s
Wall time: 57.3 s


# Генерация

In [ ]:
gpt.eval()

In [ ]:
trainer.set_variety(bias_mask, 1)

In [ ]:
%%time
gpt.to('cpu')

input_ids = tokenizer.encode('Радиолампа состоит из', return_tensors='pt').to(gpt.device)


output_sequences = gpt.generate(
    input_ids=input_ids,
    max_length=35,
    # temperature=0.7,
    #top_p=0.95,
    repetition_penalty=1.2,
    num_beams=4,
    num_return_sequences=1,
    #do_sample=True
)

generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)

print(generated_text)

Радиолампа состоит из стеклянного баллона, заполненного инертным газом, закрытого колбой, и сетевого шнура.
Радиолам
CPU times: user 18.7 s, sys: 1.59 s, total: 20.3 s
Wall time: 20.4 s


In [ ]:
%%time
g_model =  GeneratorWithExpert.GenerationWithProbs(gpt, tokenizer, bias_mask)
g_model.generate('Радиолампа состоит из', temperature = 0, variety=0.2)

CPU times: user 7.19 s, sys: 165 ms, total: 7.35 s
Wall time: 7.33 s


'Радиолампа состоит из стеклянного баллона, заполненного инертным газом. Баллон помещается в специальный держатель и закрепляется'

In [ ]:
# Генерация с экспертом
start = 'Радиолампа состоит из'
generator_with_expert = GeneratorWithExpert.GPTGenerator(expert=hmm_expert, model=gpt, tokenizer=tokenizer, device='cuda:0')
end = generator_with_expert.Generate(start, expert_w=0.6, max_len=40, num_seq=1, temperature=0.1)[0]

print(start + end)

Радиолампа состоит из стеклянного баллона, заполненного инертным газом (неоном) при небольшом давлении. Баллон наполнен инертным газом (неоном) при небольшом давлении.
Ради


# GPT Эксперт

In [ ]:
path = 'ai-forever/rugpt3small_based_on_gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(path)
gpt_ex_m = GPT2LMHeadModel.from_pretrained(path)
gpt_ex_m = gpt_ex_m.to('cuda')

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

In [ ]:
%%time
# Обучаем все слои проекции эксперта
trainer = GPTProjectionTrainer(gpt_ex_m, tokenizer)
trainer.train([text_data],bias_mask, num_train_epochs = 40, last_k=12, output_dir='s', variety=0.1)

Step,Training Loss


CPU times: user 10.3 s, sys: 255 ms, total: 10.6 s
Wall time: 12.6 s


In [ ]:
# Создание эксперта из gpt

class GPTExpert(ExpertModel):

      def __init__(self, gpt_ex_m, len_v) -> None:
         super().__init__()
         self.gpt = gpt_ex_m
         self.len_v = len_v
         self.gpt.eval()

      def get_bias(self, tokens):
        """Вычисление bias из вероятностной модели"""
        tokens_ = torch.tensor([tokens]).to(self.gpt.device)

        with torch.no_grad():
          outp = self.gpt(tokens_)
        return outp['logits'][0][-1].cpu().detach().numpy()[:self.len_v]

In [ ]:
gpt_ex_m.eval()
gpt_expert = GPTExpert(gpt_ex_m, 50257)

In [ ]:
# Генерация экспертом
generate("<s> Радиолампа состоит из", gpt_expert, tokenizer)

'<s> Радиолампа состоит из анода и катода, и потому называется катодом.\nЭлектронная лампа имеет два электрода — катод и сетку. Катод разог'

In [ ]:
start = 'Радиолампа состоит из '
generator_with_expert = GeneratorWithExpert.GPTGenerator(expert=gpt_expert, model=gpt, tokenizer=tokenizer, device='cuda:0')
end = generator_with_expert.Generate(start, expert_w=0.4, max_len=40, num_seq=1, temperature=0.1)[0]

print(start + end)

Радиолампа состоит из 
электрода, катода, сетки, колбы,
и небольшого количества газа, заполняющего её.
Радиолампы
Радиолампы
Радиолам


# Без тюнинга основной GPT

In [ ]:
# Используем gpt-large без тюнинга, только с экспертом
path = 'ai-forever/rugpt3large_based_on_gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(path)
gpt_L = GPT2LMHeadModel.from_pretrained(path)
gpt_L.to('cuda')

In [ ]:
gpt_expert = GPTExpert(gpt_ex_m, 50257)

In [ ]:
%%time
start = 'Радиолампа состоит из '
generator_with_expert = GeneratorWithExpert.GPTGenerator(expert=gpt_expert, model=gpt_L, tokenizer=tokenizer, device='cuda:0')
end = generator_with_expert.Generate(start, expert_w=0.6, max_len=70, num_seq=1, temperature=0.1, rp=1.5)[0]

print(start + end)

Радиолампа состоит из 
электронного блока, в котором размещены два электрода — катод и анод. Электроды располагаются на аноде и катоде, и при помощи электрического поля создают ток в цепи анода. Электроды подключаются к аноду через резисторы, и ток в цепи анода передается на анод. Если подать на анод перемен
CPU times: user 3.33 s, sys: 11.7 ms, total: 3.35 s
Wall time: 3.43 s
